In [ ]:
#%load_ext nb_black

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import zscore
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from mpl_toolkits import mplot3d

%matplotlib inline

# TODO: Kolommen van int omzetten naar "Int64" zodat np.NaN kan worden opgeslagen ipv 0 als placeholder
# TODO: NaN-types als laatste verwijderen na alle cleanup
# TODO: Negatieve waardes beter filteren op één regel

# TODO: Correlatiematrix toevoegen bij het selecteren van de kolommen
# TODO: Holoviews implementeren
# TODO: RMSE toelichten

# In hoeverre is het mogelijk om de budget-omzet verhouding te voorspellen?
Ook voor deze onderzoeksvraag zullen de zeven stappen van het Data Science-proces worden toegepast. 
Stappen 1, 2 en 3 zullen in grote maten hetzelfde zijn als bij de overige twee onderzoeksvragen.
### Het Data Science-proces:
1. Data collection
2. Data processing (ook wel data munging)
3. Data cleaning
4. Data exploration & analysis
5. Model building
6. Visualization
7. Communication

## 1. Data Collection
Voor het beantwoorden van deze onderzoeksvraag zal alleen gebruik worden gemaakt van `movie.csv`.
Om te zien of de dataset `movie.csv` goed is ingeladen, worden de eerste vijf rijen getoond:

In [ ]:
df_movies = pd.read_csv("../../data/movie.csv")
print(f"In totaal zijn er {len(df_movies)} films.\n")
df_movies.head()

Te zien is dat de data goed wordt ingelezen en de waardes in de juiste kolommen komen te staan.
Verder zijn er al NaN-waardes te zien en staan veel gegevens opgeslagen als floats.

***

## 2. Data Processing
Ook de Data Processing is grotendeels voor ons gedaan. De data is goed opgeslagen in een `.csv`-bestand
en is zojuist al ingelezen als een _Pandas_ DataFrame.

Verder rest ons nog de volgende stappen om de Data Processing te voltooien:
1. Het selecteren van de gewenste kolommen

2. Het aanpassen van onduidelijke kolomnamen
3. Het aanpassen van de datatypen

4. Het berekenen van de budget-gross verhouding van iedere film
5. Het berekenen van de Z-score-afstand tussen budget en film 
6. Het berekenen van de Z-scores van de feature-kolommen

Alvorens de gewenste kolommen kunnen worden geselecteerd, moet worden gekeken welke kolommen uberhaubt relevant zijn. Pandas heeft een hele gemakkelijke functie om voor een compleet DataFrame alle correlaties uit te rekenen. De correlaties worden standaard volgens Pearsons correlatiecoëfficient berekend. Vervolgens kan het correlatiematrix gemakkelijk worden geplot met Seaborn. Seaborn maakt het mogelijk om een snel overzicht te krijgen van de correlaties.

Uiteraard is het pas bij de _Data Exploration_ van belang om de data volledig te doorgronden, echter moet tijdens de _Data Processing_ al worden gekozen welke kolommen gebruikt gaan worden. Aangezien Data Science een iteratief proces is kan tijdens een later stadium ervoor worden gekozen om meer kolommen te behouden.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(df_movies.corr(), annot=True, linewidths=.05, ax=ax, square=True)
plt.show()

Na het bestuderen van het geplotte correlatiematrix blijkt dat `num_voted_users`, `num_user_for_reviews` en `num_critic_for_reviews` de grootste correlatie hebben met `budget` en `gross`. De twee laatstgenoemden zijn uiteraard vereisten voor het beantwoorden van de onderzoeksvraag. De verhouding tussen het budget en de omzet moet immers worden voorspeld.

Verder selecteren we nog de kolom `title_year`. Dit zou eventueel later van pas kunnen komen voor het sorteren of visualiseren van de data. Zoals eerder genoemd zou het mogelijk kunnen zijn dat in een volgende iteratie blijkt dat er meer kolommen nodig zijn.

In [ ]:
# Selecteren gewenste kolommen
df_movies = df_movies[["movie_title", "budget", "gross",
                       "title_year", "num_voted_users",
                       "num_user_for_reviews", "num_critic_for_reviews"]]

# Onduidelijke kolomnamen aanpassen
df_movies.rename(
    columns={
        "movie_title": "Movie title",
        "budget": "Budget",
        "gross": "Gross",
        "title_year": "Release year",
        "num_voted_users": "Number of voted users",
        "num_user_for_reviews": "Number of user reviews",
        "num_critic_for_reviews": "Number of critics"}, inplace=True,)

print(f"Kolommen:")
df_movies.columns.tolist()

Integers zijn een stuk sneller dan floats. Het komt de efficiëntie en netheid ten goede om zo veel mogelijk kolommen naar integers
om te zetten.

NaN-types kunnen echter niet worden opgeslagen in kolommen met het datatype `int32`. Als oplossing is ervoor gekozen om alle NaN-types om te zetten naar integers. De meest logische keuze zal 0 zijn.

Het datatype `Int64` geeft wel de mogelijkheid om NaN-types samen met integers op te slaan. Echter is het `Int64`-datatype nog experimenteel en is de afweging gemaakt om dit datatype niet te gebruiken.

[Bron: Nullable integer data type](https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html)


In [ ]:
# Datatypen aanpassen: Floats naar integers omzetten
df_movies.fillna(0, inplace=True)
df_movies = df_movies.astype({"Budget": int,
                              "Gross": int,
                              "Number of voted users": int, 
                              "Number of user reviews": int,
                              "Number of critics": int})

# Datatypes aanpassen: De kolom `Release year` omzettten van float naar het datetime-datatype
df_movies["Release year"] = pd.to_datetime(df_movies["Release year"], format="%Y", errors="coerce")
df_movies.dtypes

Het beantwoorden van de onderzoeksvraag vraagt naar het voorspellen van de budget/omzet-verhouding.
De kolom `Budget Gross verh` bevat de mate waarin `budget` en `gross` zich verhouden. Dit werkt middels de simpele formule $\frac {Gross} {Budget}$. Een waarde groter dan `1` betekend winst. Er wordt dan meer omzet gedraaid dan dat er budget was de film. Alles onder de `1` betekend verlies. Hierbij is de omzet lager dan het budget.

`Budget Gross Zscore afst` houdt het (absolute) verschil tussen de Zscore van het budget
en de gross in. Een klein voorbeeld ter illustratie:
> - Het budget van een film heeft een Z-score van 1
> - De omzet van dezelfde film heeft een Z-score van 4
>
> De `Budget Gross Zscore afst` van deze film bedraagt $(|4-1|)=$ $3$  
> Omdat de afstand minder dan 6 is kan worden geconcludeerd dat het onderlinge verschil niet significant en de film mag blijven in de dataset.
  
Het verwijderen van waardes met een absolute Z-score afstand van meer dan 6 zal later gebeuren tijdens de _Data Cleaning_. Door deze significante waardes te verwijderen uit de dataset zullen betere voorspellingen mogelijk worden. Uitschieters kunnen de precisie van een model ernstig verslechteren.

In [ ]:
# Verhouding tussen budget en gross berekenen en opslaan in nieuwe kolom
df_movies["Budget Gross verh"] = df_movies["Gross"] / df_movies["Budget"]

# Zscores berekenen van Budget en Gross.
df_movies["Budget Gross Zscore afst"] = abs(zscore(df_movies[["Budget"]]) - zscore(df_movies[["Gross"]]))

# Z-scores berekenen van de mogelijke features
df_movies["Number of voted users Zscore"] = abs(zscore(df_movies[["Number of voted users"]]))
df_movies["Number of user reviews Zscore"] = abs(zscore(df_movies[["Number of user reviews"]]))
df_movies["Number of critics Zscore"] = abs(zscore(df_movies[["Number of critics"]]))

Het DataFrame `df_movies` ziet er nu als volgt uit:

In [ ]:
df_movies.head()

Alle kolommen van het DataFrame hebben nu het juiste datatype.
Onder andere `Budget` en `Gross` bestaan uit integers en `Release year` bestaat uit het `datetime` datatype.
Echter zijn er nog wel 0-, NaN, en NaT-waardes zichtbaar. Deze rijen dienen tijdens de _Data
Cleaning_ nog te worden opgeschoont.


***

## 3. Data Cleaning
Nu de data in de juiste kolommen met de juiste datatypes is opgeslagen kunnen we de data gaan opschonen.
Na het uitvoeren van de volgende vijf stappen hebben we een DataFrame dat klaar voor analyse is.
1. Het verwijderen van dubbele `Movie titles`
2. Het omzetten van onrealistische waardes naar 0
3. Het verwijderen van rijen met een `Budget Gross zscore verh` van meer dan 6
4. Het verwijderen van films met een budget of gross van 0
5. Het verwijderen van NaN-types


In [ ]:
print(f"In totaal zijn er vóór de Data Cleaning {len(df_movies)} films.")

# Dubbele titels verwijderen
df_movies.sort_values("Release year", inplace=True)  # Sorteren op uitgavejaar
df_movies.drop_duplicates(subset="Movie title", keep="last", inplace=True)  # Alleen meest recente versie blijft bewaard
print(f"In totaal zijn er {len(df_movies)} films zonder duplicaten over.")

# Onrealistische waardes omzetten
print(f"In totaal zijn er {df_movies[['Budget', 'Gross']].lt(0).sum().sum()} films met een negatief Budget of Gross.")
print(f"In totaal zijn er {df_movies[['Budget']].gt(400_000_000).sum().sum()} films met een budget hoger dan de duurste film ooit.")
print(f"In totaal zijn er {df_movies[['Budget', 'Gross']].gt(2_800_000_000).sum().sum()} films met een hogere omzet dan de meest succesvolle film ooit.")
print(f"In totaal zijn er {df_movies['Budget Gross Zscore afst'].gt(6).sum().sum()} films met een extreem verschil tussen budget en gross.")

# Negatieve waardes omzetten
df_movies = df_movies[(df_movies[["Budget", "Gross"]] >= 0).all(axis=1)]

# 'Te dure' films verwijderen
df_movies = df_movies[(df_movies["Budget"] < 400_000_000)]

# Verwijderen films met Z-score afstand van meer dan 6
df_movies = df_movies[(df_movies["Budget Gross Zscore afst"] <= 6)]

# Verwijderen films met budget of gross van 0
df_movies = df_movies[(df_movies[["Budget", "Gross"]] != 0).all(axis=1)]

# NaN-types verwijderen uit de titels
df_movies.dropna(inplace=True)
print(f"In totaal blijven er {len(df_movies)} films over.")

Na _stap 3. Data Cleaning_ ziet het DataFrame er als volgt uit:

In [ ]:
df_movies.head()

Het DataFrame is nu volledig opgeschoond en klaar voor de _Data Exploration & Analysis_.

Het DataFrame bevat alleen maar nog unieke waardes voor `Movie title`.
Door het DataFrame te sorteren op `Release year` en vervolgens de laatste waarde te behouden, wordt
de meest recente uitgave van een film bewaard. Te zien is dat maar liefst 126 films dubbel
in de dataset voorkwamen.

Het omzetten van alle onrealistische waardes van `Budget` en `Gross` worden vervolgens omgezet naar `0`. Onrealistische waardes van `Budget` en `Gross` zijn onder andere negatieve waardes. Andere onrealistische waardes zijn films met een `Budget` en/of `Gross` van meer dan 2.8 miljard. Er is immers nog nooit een film* geweest die meer dan 2.8 miljard opbracht.
Verder werden alle films met een `Budget` van meer dan 400 miljoen gedropped, de duurste film ooit
kostte niet meer dan 400 miljoen. Door deze stap worden enkele extreme outliers gefilterd uit
de data wat de voorspellingen en visualisatie ten goede zal komen. 

[Bron: Lijst van succesvolste films](https://nl.wikipedia.org/wiki/Lijst_van_succesvolste_films )

Uiteindelijk worden alle films met een `Budget` of `Gross` van 0 verwijderd uit het DataFrame.

Zoals beschreven bij de _Data Processing_ worden alle waardes met een significant verschil tussen de Z-scores van `Budget` en `Gross` verwijderd. Zie voor meer toelichting de documentatie bij _Data Processing_.

***

## 4. Data Exploration & Analysis

Nu de data is geprepareerd, kan begonnen worden aan de eerste verkenning. Door middel van
describe krijgen we in één oogopslag een duidelijk beeld van het DataFrame.

In [ ]:
df_movies.describe(include="all")

Enkele verwachtingen:
1. Het meerendeel van de films zal winst maken
2. Nieuwere films zullen relatief meer winst maken dan oudere films
3. Het budget van relatief oudere films zal lager zijn dan dat van relatief nieuwe films 

Enkele eerste observaties:
- Het DataFrame bevat nog 3772 (74.9%) van de originele 5043 rijen 
- `Movie title` bevat zoals beoogd alleen maar nog unieke waardes.
- Gemiddeld brengt een film 49 miljoen op en is er een budget van 37 miljoen. Echter heeft
 het budget een standaard deviatie van 42 miljoen en de omzet een standaard deviatie van 65 miljoen.
- Het `Release year` 2002 komt met 189x het vaakst voor
- Het DataFrame bevat films die uitgegeven zijn tussen 1920 en 2016. Een interval van bijna 100 jaar.


In de volgende scatterplot zijn het budget en de omzet van films geplot.
Dit geeft een eerste beeld van de spreiding van de data en de relatie tussen het budget en de omzet.
Er wordt een sample van 350 (+- 10%) stuks genomen om de data beter te visualiseren.

Tevens wordt de baseline al getekend. Deze wordt vastgesteld op $y = x$ oftewel het break-even point.
Alles films onder de lijn hebben verlies geleden, allen boven de lijn maakten winst. De baseline
wordt gebruikt om vast te stellen een geproduceerd model wel echt beter is dan geen model.

In [ ]:
fig, ax = plt.subplots()
df_movies_sample = df_movies.sample(350, random_state=42)
sns.scatterplot("Budget", "Gross", data=df_movies_sample)

coords = [np.min([ax.get_xlim(), ax.get_ylim()]),
          np.max([ax.get_xlim(), ax.get_ylim()])]

ax.plot(coords, coords, zorder=0, color='black')
plt.show()

Een mogelijke functie om, in plaats van een selectie, alle films te analyseren is `jointplot`.
Jointplots geven op een snelle manier de verspreiding maar vooral dichtheid van punten weer.
Er is goed te zien dat de meeste films onder een budget van één miljoen blijven en 
dat veruit de meeste films onder een omzet van twee miljoen blijven.
Er valt zelfs al een kleine positieve correlatie te spotten.

Om de plot overzichtelijk te houden is gekozen om de dichtheid weer te geven.
Het grote aantal films kan, zoals we hierboven al zagen, tot een onoverzichtelijke plot leiden.
Credits: seaborn.pydata.org/generated/seaborn.jointplot.html

In [ ]:
sns.jointplot("Budget", "Gross", data=df_movies, kind="kde", space=0)
plt.show()

In de vorige jointplot waren individuele outliers niet te zien.
Boxplots zijn uitermate geschikt om deze wel weer te geven.

De omzet van films is veel verspreider dan het budget van films. Tevens zijn
er bij de omzet veel meer extreme outliers te zien.
Door het grote aantal uitschieters kunnen zelfs boxplots heel onoverzichtelijk worden.

In [ ]:
df_movies[["Budget", "Gross"]].boxplot(grid=False)
plt.show()

Een laatste poging om ook de outliers in kaart te brengen wordt gedaan door twee
histogrammen te plotten. Bovenop de histogrammen worden streepjes getekend. Ieder streepje is een
film. De outliers zijn nu zichtbaar geworden. Direct valt te zien dat het budget
twee extreme uitschieters heeft bij drie en resp. vier miljoen.
Gross heeft uitschieters die door gaan tot zo'n ongeveer 8 miljoen.

Ook hier wordt weer duidelijk dat het meerendeel van de films het budget onder de één miljoen houdt
en de opbrengst vooral onder de twee miljoen blijft. Er zijn slechts enkele films met een extreme
verhouding tussen het budget en de omzet.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 3), sharex="row", sharey="row")
sns.distplot(df_movies["Budget"], rug=True, kde=False, ax=axes[0], color="r")
sns.distplot(df_movies["Gross"], rug=True, kde=False, ax=axes[1], color="g")


sns.distplot(df_movies["Budget Gross verh"], rug=True, kde=False, ax=axes[0], color="b")
sns.distplot(df_movies["Number of critics"], rug=True, kde=False, ax=axes[1], color="c")
plt.show()

Pandas heeft een hele handige methode ingebouwd voor DataFrames. Deze methode berekend direct de
correlatie tussen iedere combinatie van twee variabelen. Standaard wordt de correlatie
door middel van Pearsons correlatiecoëfficiënt berekend.

Volgens Pearsons correlatiecoëfficiënt blijkt dat er een correlatie van 0.64 is tussen Budget en
Gross. Verder geven `Number of voted users`, `Number of user reviews` en `Number of critics` ook redelijke correlaties bij
Budget en ook Gross. Echter willen we voor het beantwoorden van de onderzoeksvraag de correlatie tussen
de verschillende features en `Budget Gross verh` weten.

In [ ]:
df_movies.corr()

## 5. Model Building
Om een beeld te krijgen van de verhouding tussen het budget en de omzet bouwen we een klein
Linear Regressie-model. Voor de x-as wordt het budget geselecteerd en voor de y-as de omzet. Vervolgens wordt
de data gesplit in een test- en trainset. Aangezien er redelijk veel data tot de beschikking is,
wordt er getraind met een 80/20 verhouding.

In [ ]:
X = df_movies[["Budget"]]
y = df_movies["Gross"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)
print(f"Aantal trainwaarden: {len(X_train):5}")
print(f"Aantal testwaarden: {len(y_test):6}")

Vervolgens dient het model te worden aangemaakt en getraind. Zodra dit model getraind is kan de
formule $Y = AX + B$ worden ingevuld. Dit maakt het mogelijk om ieder mogelijk budget $x$ in
te vullen waarna $y$ zal resulteren in de voorspelde omzet.

In [ ]:
linReg = LinearRegression()
linReg.fit(X_train, y_train)
print(f"Formule van de lijn:\nY = {linReg.coef_[0]:.4f} * X + {linReg.intercept_:.4f}")

Zodra het model is gemaakt en getraind dient het te worden gevalideerd. Belangrijke 
prestatieindicatoren voor Lineare Regressie zijn de $R^2$ en de Root Mean Squared Error (RMSE).
 

In [ ]:
y_pred = linReg.predict(X_test).reshape(-1, 1)
print(f"R2 score: {linReg.score(y_pred, y_test):.4f}")
print(f"Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred)):_.4f}\n")  # Werkt

Om bovenstaand model weer te geven wordt bovenop een scatterplot de vastgestelde lineare formule getekend.
Deze lijn wordt in het rood weergegeven. In het zwart is weer de baseline te zien.

Te zien is dat de vastgestelde formule, de y-as
bij een x-waarde van 0, hoger snijdt dan de baseline. Aangezien $A$ kleiner dan 1 is, zal
de lijn na verloop van tijd de baseline snijden.

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(X_test["Budget"], y_test)
plt.plot(X_test, y_pred, color="red")

coords = [np.min([ax.get_xlim(), ax.get_ylim()]),
          np.max([ax.get_xlim(), ax.get_ylim()])]

ax.plot(coords, coords, zorder=0, color='black', alpha=0.5)
plt.show()

## Tussentijdse evaluatie
Na het evalueren van bovenstaand model en het opnieuw bestuderen van de onderzoeksvraag zijn
we tot de conclusie gekomen dat bovenstaand model iets anders doet dan waar de onderzoeksvraag
naar vraagt. Met bovenstaand model kan, tot op zekere hoogte, de omzet worden voorspeld bij
een bepaald budget.

Waar de onderzoeksvraag echter naar vraagt is het voorspellen van de budget/omzet-verhouding.
Aan ons de taak om een goede combinatie van features te selecteren waardoor de 
budget/omzet-verhouding kan worden voorspeld. Tijdens de dataprocessing is een correlatiematrix
getoond waaruit bleek dat `Number of voted users`, `Number of user reviews` en `Number of critics`
de beste correlatie vertonen met `Budget` en/of `Gross`.

Er wordt een nieuw model opgesteld met deze set features als X. Verder wordt `Budget Gross verh`
als Y genomen.

In [ ]:
X = df_movies[["Number of voted users",
               "Number of user reviews",
               "Number of critics"]]
y = df_movies["Budget Gross verh"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)
print(f"Aantal trainwaarden: {len(X_train):5}")
print(f"Aantal testwaarden: {len(y_test):6}")

Ook zoals bij het vorige model wordt het getraind en zal de formule van de lijn worden berekend.

In [ ]:
linReg = LinearRegression()
linReg.fit(X_train, y_train)
print(f"Formule van de lijn:\nY = {linReg.coef_[0]:.4f} * X + {linReg.intercept_:.4f}")

Ten slotte worden de R2 Score en de RMSE ook voor dit model berekend.
Een R2 score van 0.0592 is extreem laag. De drie geselecteerde features samen lijken geen
correlatie te hebben met de budget/omzet-verhouding. Deze features hadden individueel
tijdens de Data Exploration echter wel enige maat van correlatie met budget en/of omzet.

In [ ]:
y_pred = linReg.predict(X_test)
print(f"R2 score: {linReg.score(X_test, y_test):.4f}")
print(f"Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred)):_.4f}\n")

In [ ]:
X_combinaties = [["Number of voted users", "Number of user reviews", "Number of critics"],
                 
                 ["Number of voted users", "Number of user reviews"],
                 ["Number of voted users", "Number of critics"],
                 ["Number of user reviews","Number of critics"],
                 
                 ["Number of voted users"],
                 ["Number of user reviews"], 
                 ["Number of critics"]]

for features in X_combinaties:
    X = df_movies[features]
    y = df_movies["Budget Gross verh"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)
    linReg.fit(X_train, y_train)
    y_pred = linReg.predict(X_test)
    
    print(f"Conbinatie: {X.columns.values}")
    print(f"R2 score: {linReg.score(X_test, y_test):22.4f}")
    print(f"Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred)):_.4f}\n")

In [ ]:
X_combinaties = [["Number of voted users Zscore"],
                 ["Number of user reviews Zscore"], 
                 ["Number of critics Zscore"]]

fig, ax = plt.subplots()
ax.plot((0, 10), (0, 10), zorder=0, color='black', label="Baseline")

for features in X_combinaties:
    X = df_movies[features]
    y = df_movies["Budget Gross verh"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)
    linReg.fit(X_train, y_train)
    y_pred = linReg.predict(X_test)
    ax.plot(X_test, y_pred, label=features[0])

plt.legend()
plt.show()

In [ ]:
from mpl_toolkits import mplot3d
%matplotlib inline

X_combinaties = [["Number of voted users Zscore", "Number of user reviews Zscore"],
                 ["Number of voted users Zscore", "Number of critics Zscore"],
                 ["Number of user reviews Zscore","Number of critics Zscore"]]

for features in X_combinaties:
    X = df_movies[features]
    y = df_movies["Budget Gross verh"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)
    linReg.fit(X_train, y_train)
    
    y_pred = linReg.predict(X_test)
    x_line = np.linspace(0, 10, 100)
    y_line = linReg.coef_[0] * x_line + linReg.intercept_
    z_line = linReg.coef_[1] * x_line + linReg.intercept_
    
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    plt.title(f"{features[0]}\n{features[1]}")
    
    ax.scatter3D(X_test[features[1]], X_test[features[0]], y_pred)
    ax.plot3D(x_line, y_line, z_line, color="red")

plt.show()

In [ ]:
import holoviews as hv
from holoviews import dim, opts

hv.extension('matplotlib')
hv.Scatter3D((X_test[features[1]], X_test[features[0]], y_pred))